# An introduction to Polars

This demo shows off the basics of polars.

[Polars User Guide](https://pola-rs.github.io/polars-book/user-guide/index.html)

[Polars API](https://pola-rs.github.io/polars/py-polars/html/reference/)

In [ ]:
# Install polars
!pip install polars

In [ ]:
# Imports

import polars as pl
import pandas as pd
import time

# Load data from a csv

Batting Table in Baseball Database from Project 3 of DS 250

[Data dictionary](https://query.data.world/s/c3tdzy3tscdeqqajz3svl5w5yjbqnu)


In [ ]:
# Loading data from csv

df = pl.read_csv("https://query.data.world/s/ikeqerehorky64m2il2vp4m4e3jrea")

In [ ]:
# Inspect some dataframe properties

print("Dataframe Schema")
print(df.schema)

print("-------------------------------------------------------------------------------------")

print("Data shape")
print(df.shape)

In [ ]:
# Inspect the first few rows

df.head()

# Select, Filter,  and New Columns

In [ ]:
# Use select to grab a subset of columns

df.select(
    ["playerID", "yearID", "H"]
)

In [ ]:
# You can also do computations inside select.

df.select(
    [
        pl.count().alias("Num Rows"),
        pl.col("G").sum().alias("Sum of G")
    ]
)

In [ ]:
# Use filter to grab a subset of records that satisfy some condition

df.filter(
    pl.col("yearID")==2000
)

In [ ]:
# Use with_columns to create new columns from existing ones. Serves similar purpose as mutate in R.

df.with_columns(
    [
        (pl.col("HR")+1).alias("Extra HR"),
        (pl.col("HR")-1).alias("Fewer HR")
    ]
)

# Group by aggregations

In [ ]:
# Use groupby(...).agg(...) for crosstab aggregations.

df.groupby("yearID").agg(
    pl.col("HR").max().alias("Max HR")
)

In [ ]:
# Chain operations

# Adds a batting average column for all players with at least one appearance at bat.
# Then computes the average batting average by year.

df.select(
    "yearID",
    "AB", # at bat
    "H" # hits
).filter(
    pl.col("AB") > 0
).with_columns(
    (pl.col("H") / pl.col("AB")).alias("Batting Avg")
).groupby(
    "yearID"
).agg(
    pl.col("Batting Avg").mean().alias("Avg Batting Avg")
)

# Missing Data Exercise

You have probably learned a few different methods for dealing with missing data such as dropping the missing values or replacing missing values by the mean or median value of the existing values. In small groups, 

- decide on a strategy for handling the missing values in the RBI column.
- search for a way to implement the solution looking through the Polars User Guide or API
- implement your strategy


In [ ]:
# Missing Data, represented as "null" in polars

df["RBI"].null_count()

In [ ]:
# Implement your method for handling missing data here

nullless_rbi = ...

In [ ]:
# This count should return 0.

nullless_rbi["RBI"].null_count()

# Eager vs Lazy

Pandas is eager and Spark is lazy. Polar supports both evaluation paradigms.

Excerpt from Polars user guide.
>Eager evaluation is where code is evaluated as soon as you run the code. Lazy evaluation is where running a line of code means that the underlying logic is added to a query plan rather than being evaluated.

Building a query plan allows for more efficient query evaluation on large-scale programs.


In [ ]:
# Simply add lazy() to the start of your query.

df.lazy().select(
    "yearID",
    "AB", # at bat
    "H" # hits
).filter(
    pl.col("AB") > 0
).with_columns(
    (pl.col("H") / pl.col("AB")).alias("Batting Avg")
).groupby(
    "yearID"
).agg(
    pl.col("Batting Avg").mean().alias("Avg Batting Avg")
)

In [ ]:
# Use collect() to materialize the results

df.lazy().select(
    "yearID",
    "AB", # at bat
    "H" # hits
).filter(
    pl.col("AB") > 0
).with_columns(
    (pl.col("H") / pl.col("AB")).alias("Batting Avg")
).groupby(
    "yearID"
).agg(
    pl.col("Batting Avg").mean().alias("Avg Batting Avg")
).collect()

In [ ]:
# Caching

bat_avg_df = df.lazy().select(
    "yearID",
    "teamID",
    "AB", # at bat
    "H" # hits
).filter(
    pl.col("AB") > 0
).with_columns(
    (pl.col("H") / pl.col("AB")).alias("Batting Avg")
).cache()


print(
    bat_avg_df.groupby(
        "yearID"
    ).agg(
        pl.col("Batting Avg").mean().alias("Avg Batting Avg")
    )
)

print(
    bat_avg_df.groupby(
        ["yearID", "teamID"] 
    ).agg(
        pl.col("Batting Avg").mean().alias("Avg Batting Avg")
    )
)

# Arrow, Parquet: Need for Speed

[Parquet](https://parquet.apache.org/) is an "on-disk" storage alternative to formats like "csv". Parquet store data by column instead of by row.

[Arrow](https://arrow.apache.org/) is an "in-memory" column-oriented data structure.

Arrow and Parquet are very efficient.

See [10 things I hate about pandas](https://wesmckinney.com/blog/apache-arrow-pandas-internals/)

Polars is built on top of Arrow.

# REMINDER: DO NOT RUN IN CLASS

In [ ]:
# We need a bigger dataset.

# Do not run this in class!!!

bigdata = df.sample(100000000, with_replacement=True)

bigdata.write_parquet("baseball.parquet")
bigdata.write_csv("baseball.csv")

# DEMO THESE CELLS, STUDENTS WON'T BE ABLE TO RUN

In [ ]:
start = time.time()

# scan instead of read begins lazy evaluation from the get go.
df = pl.scan_parquet("baseball.parquet")

df.select(
    "yearID",
    "AB",
    "H"
).filter(
    pl.col("AB") > 0
).with_columns(
    (pl.col("H") / pl.col("AB")).alias("Batting Avg")
).groupby(
    "yearID"
).agg(
    pl.col("Batting Avg").mean().alias("Avg Batting Avg")
).collect()

elapsed = time.time() - start
print(elapsed)

In [ ]:
start = time.time()
pdf = pd.read_parquet("baseball.parquet", columns=["yearID", "AB", "H"])
pdf = pdf[["yearID", "AB", "H"]]
pdf = pdf[pdf["AB"] > 0].reset_index()
pdf["BA"] = (pdf["H"] / pdf["AB"])
pdf = pdf.groupby("yearID").BA.agg("mean")
elapsed = time.time() - start
print(elapsed)

In [ ]:
start = time.time()
pdf = pd.read_csv("baseball.csv", usecols=["yearID", "AB", "H"])
pdf = pdf[["yearID", "AB", "H"]]
pdf = pdf[pdf["AB"] > 0].reset_index()
pdf["BA"] = (pdf["H"] / pdf["AB"])
pdf = pdf.groupby("yearID").BA.agg("mean")
elapsed = time.time() - start
print(elapsed)

# Teach One Another Exercise

- Discover something new about polars. Think of a topic that you do frequently in data science and spend some time figure out whether polars supports it!